In [10]:
from pandas import read_csv
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_log_error, r2_score

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
import matplotlib.pyplot as plt

df = pd.read_csv("dataset.csv")
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


# 1
2. Si vuole predire il prezzo di vendita delle case. Ricaricare il dataset originale, eliminare
eventuali attributi inutili (giustificare la scelta) ed eventuali istanze con valori nulli. Convertire i
valori delle colonne mainroad, guestroom, basement, hotwaterheating,
airconditioning, prefarea in modo che “yes” sia sostituito con 1 e “no” con 0.
T rasformare anche i valori testuali della colonna furnishingstatus in valori numerici a piacere.
Dividere il dataset in modo che 3/4 degli elementi siano contenuti in un nuovo dataset “train” e
1/4 nel dataset “test”
.
Allenare il train con il modello LinearRegression e valutare il Mean Squared Logarithmic Error
(MSLE) e R2 sia sul dataset train sia sul dataset test. Confrontare i risultati ottenuti con quelli
ottenuti con una predizione basata sul modello SGDRegressor . Effettuare alcune considerazioni
sui risultati ottenuti. (punti 4)

In [11]:
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
mapping = {'yes': 1, 'no': 0}
for col in binary_cols:
    df[col] = df[col].map(mapping)

furnishing_map = {'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2}
df['furnishingstatus'] = df['furnishingstatus'].map(furnishing_map)

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# LINEAR REGRESSION
lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)

def safe_msle(y_true, y_pred):
    try:
        # MSLE richiede valori positivi. Se il modello predice negativi, clippiamo a 0
        y_pred_safe = np.where(y_pred < 0, 0, y_pred)
        return mean_squared_log_error(y_true, y_pred_safe)
    except:
        return np.nan

#Valutazione LR
print("--- Linear Regression ---")
print(f"Train R2:   {r2_score(y_train, y_train_pred_lr):.4f}")
r2_lr_ = r2_score(y_train, y_train_pred_lr)
print(f"Test R2:    {r2_lr_:.4f}")
print(f"Train MSLE: {safe_msle(y_train, y_train_pred_lr):.4f}")
print(f"Test MSLE:  {safe_msle(y_test, y_test_pred_lr):.4f}")

#SGD
sgd = SGDRegressor(random_state=42)
sgd.fit(X_train, y_train)

# Predizioni
y_train_pred_sgd = sgd.predict(X_train)
y_test_pred_sgd = sgd.predict(X_test)

# Valutazione SGD
print("\n--- SGD Regressor ---")
print(f"Train R2:   {r2_score(y_train, y_train_pred_sgd):.4f}")
r2_sgd_ = r2_score(y_train, y_train_pred_sgd)
print(f"Test R2:    {r2_sgd_:.4f}")
print(f"Train MSLE: {safe_msle(y_train, y_train_pred_sgd):.4f}")
print(f"Test MSLE:  {safe_msle(y_test, y_test_pred_sgd):.4f}")

--- Linear Regression ---
Train R2:   0.6839
Test R2:    0.6839
Train MSLE: 0.0391
Test MSLE:  0.0629

--- SGD Regressor ---
Train R2:   -28768507652213022720.0000
Test R2:    -28768507652213022720.0000
Train MSLE: 234.5644
Test MSLE:  234.0780


Mentre LR funziona bene con i dati non scalati in quanto usa differenze tra i quadrati, SGD fa cacare, quindi i risultati sono pessimi. Quindi dovremmo usare uno standard scaler, in questo modo area (che ha valori fino a milione) avrebbe la stessa importanza del numero di bagni.

# 2
2. Confrontare i valori di R2 ottenuti nel punto precedente con il valore di R2 che si ottiene con
una 5 Fold cross validation. (punti 1)

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores_lr = []
r2_scores_sgd = []

for i, (train_index, test_index) in enumerate(kf.split(X)):

    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    lr = LinearRegression()
    lr.fit(X_train_fold, y_train_fold)
    y_pred_lr = lr.predict(X_test_fold)
    r2_lr = r2_score(y_test_fold, y_pred_lr)
    r2_scores_lr.append(r2_lr)

    sgd = SGDRegressor(random_state=42)
    sgd.fit(X_train_fold, y_train_fold)
    y_pred_sgd = sgd.predict(X_test_fold)
    r2_sgd = r2_score(y_test_fold, y_pred_sgd)
    r2_scores_sgd.append(r2_sgd)


mean_r2_lr = np.mean(r2_scores_lr)
mean_r2_sgd = np.mean(r2_scores_sgd)

print("-" * 30)
print(f"MEDIA R2 Linear Regression con fold: {mean_r2_lr:.4f}")
print(f"Test R2 Linear Regression:    {r2_lr_:.4f}")
print(f"MEDIA R2 SGD Regressor con fold:     {mean_r2_sgd:.4f}")
print(f"Test R2 SGD Regressor:    {r2_sgd_:.4f}")

------------------------------
MEDIA R2 Linear Regression con fold: 0.6319
Test R2 Linear Regression:    0.6839
MEDIA R2 SGD Regressor con fold:     -79790809514932502528.0000
Test R2 SGD Regressor:    -28768507652213022720.0000


# 3
Trovare i parametri migliori di SGDRegressor , agendo sui parametri loss e penalty.
Scegliere alcuni valori da testare e riportare i valori di MSLE e R2 ottenuti con la migliore
configurazione, confrontare questi valori con quelli ottenuti al punto 1. (punti 3)

In [13]:
param_grid = {
    'loss' : ['squared_error', 'epsilon_insensitive', 'huber'],
    'penalty' : ['l1', 'l2'],
}
gs = GridSearchCV(SGDRegressor(), param_grid, cv=5, scoring='r2')
gs.fit(X_train, y_train)

best_sgd = gs.best_estimator_
print(f"Migliori parametri trovati: {gs.best_params_}")
best_params = gs.best_params_
y_pred_tuned = best_sgd.predict(X_test)
tuned_r2 = r2_score(y_test, y_pred_tuned)
tuned_msle = safe_msle(y_test, y_pred_tuned)

print(f"\n--- RISULTATI SGD TUNED ---")
print(f"R2 Score: {tuned_r2:.4f}")
print(f"MSLE:     {tuned_msle:.4f}")

print(f"\n--- CONFRONTO ---")
if tuned_r2 > r2_sgd_:
    print("La nuova configurazione ha migliorato il risultato (o ridotto il disastro). ")
else:
    print("Non c'è stato miglioramento significativo.")

Migliori parametri trovati: {'loss': 'epsilon_insensitive', 'penalty': 'l1'}

--- RISULTATI SGD TUNED ---
R2 Score: 0.2030
MSLE:     0.1488

--- CONFRONTO ---
La nuova configurazione ha migliorato il risultato (o ridotto il disastro). 


# 4
Studiare la correlazione tra le feature del dataset, creare un dataframe che contiene, oltre
alla colonna target, le 5 feature più correlate (positivamente) al target. Ripetere la predizione
sul nuovo dataset e verificare se il MSLE ottenuto con LinearRegression e SGDRegressor migliora
(punti 3).

In [14]:
correlation_matrix = df.corr()
top_corr_features = correlation_matrix['price'].sort_values(ascending=False)[0:6].index.tolist()
new_df = df[top_corr_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)

r2_lr = r2_score(y_test, y_pred_lr)
msle_lr = safe_msle(y_test, y_pred_lr)

print("--- RISULTATI LINEAR REGRESSION ---")
print(f"R2:   {r2_lr:.4f}")
print(f"R2(old):   {r2_lr_:.4f}")
print(f"MSLE: {msle_lr:.4f}")

r2_sgd_top = r2_score(y_test, y_pred_sgd)
msle_sgd_top = safe_msle(y_test, y_pred_sgd)

print("\n--- RISULTATI SGD REGRESSOR ---")
print(f"R2:   {r2_sgd_top:.4f}")
print(f"R2(old):   {r2_sgd_:.4f}")
print(f"MSLE: {msle_sgd_top}")

--- RISULTATI LINEAR REGRESSION ---
R2:   0.6615
R2(old):   0.6839
MSLE: 0.0629

--- RISULTATI SGD REGRESSOR ---
R2:   -19101720069635112960.0000
R2(old):   -28768507652213022720.0000
MSLE: 234.07797159579576


# 5
Considerare il dataset usato al punto 1, creare una pipeline in cui al dataset normalizzato si
aggiunga una colonna che contiene i valori della colonna area discretizzati in 5 intervalli. La
pipeline deve applicare il modello SGDRegressor con i parametri migliori trovati al punto 3.
Valutare MSLE e R2 della predizione. (punti 3)

In [15]:
df = read_csv("dataset.csv")

binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
mapping = {'yes': 1, 'no': 0}
for col in binary_cols:
    df[col] = df[col].map(mapping)

furnishing_map = {'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2}
df['furnishingstatus'] = df['furnishingstatus'].map(furnishing_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), X_train.columns),

        ('disc_area', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform'), ['area'])
    ],
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(random_state=42, max_iter=100000, **best_params))
    ]
)

pipeline.fit(X_train, y_train)
y_pred_v2 = pipeline.predict(X_test)

r2_v2 = r2_score(y_test, y_pred_v2)
msle_v2 = safe_msle(y_test, y_pred_v2)

print(f"--- RISULTATI PIPELINE (Scaling + Area Discretizzata) ---")
print(f"R2 Score: {r2_v2:.4f}")
print(f"MSLE:     {msle_v2:.4f}")


--- RISULTATI PIPELINE (Scaling + Area Discretizzata) ---
R2 Score: -5.0043
MSLE:     37.8575


Ha questi risultati del cazzo perché i params sono i migliori per i valori non scalati, ora che li abbiamo scalati non funziona più

# 6
Aggiungere in coda alla pipeline la funzione SelectKBest. Utilizzare la funzione di
gridSearchCV per selezionare il K migliore e anche gli intervalli migliori in cui discretizzare i
valori di area. Ignorare eventuali warning (punti 3).

In [16]:
import warnings

warnings.filterwarnings('ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), X_train.columns),
        ('disc_area', KBinsDiscretizer(encode='ordinal', strategy='uniform'), ['area'])
    ]
)

pipeline_grid = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('selector', SelectKBest(score_func=f_regression)),
        ('model', SGDRegressor(random_state=42, max_iter=10000)) # max_iter alto per sicurezza
    ]
)

# Sintassi: nome_step__nome_sottostep__parametro
param_grid = {
    # Testiamo diversi K per SelectKBest (step 'selector')
    'selector__k': [5, 10, 15, 'all'],
    # Testiamo diversi intervalli per l'area (step 'preprocessor' -> 'disc_area')
    'preprocessor__disc_area__n_bins': [3, 5, 7, 10]
}

grid_search = GridSearchCV(pipeline_grid, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)
print(f"\nMigliori parametri trovati: {grid_search.best_params_}")
print(f"Miglior R2 (da Cross Validation): {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_
test_r2 = best_model.score(X_test, y_test)
print(f"R2 finale sul Test Set: {test_r2:.4f}")


Migliori parametri trovati: {'preprocessor__disc_area__n_bins': 3, 'selector__k': 15}
Miglior R2 (da Cross Validation): 0.6145
R2 finale sul Test Set: 0.6567


Considerare il dataset usato al punto 1, creare una nuova pipeline in cui gli attributi
bedrooms, bathrooms e stories sono discretizzati in 2 intervalli, l’attributo area è
trasformato con uno StandardScaler e tutti gli altri attributi sono lasciati invariati. La pipeline
deve applicare il modello SGDRegressor con i parametri migliori trovati al punto 3. Valutare i
risultati ottenuti e confrontarli con quelli ottenuti al punto 5. (punti 3).

In [17]:
df = read_csv("dataset.csv")

binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
mapping = {'yes': 1, 'no': 0}
for col in binary_cols:
    df[col] = df[col].map(mapping)

furnishing_map = {'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2}
df['furnishingstatus'] = df['furnishingstatus'].map(furnishing_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
preprocessor = ColumnTransformer(
    transformers=[
        ('disc_rooms', KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='uniform'), ['bedrooms', 'bathrooms', 'stories']),
        ('scale_area', StandardScaler(), ['area']),
    ], remainder='passthrough'
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(random_state=42, max_iter=100000, **best_params))
    ]
)


pipeline.fit(X_train, y_train)
y_pred_v3 = pipeline.predict(X_test)

r2_v3 = r2_score(y_test, y_pred_v3)
msle_v3 = safe_msle(y_test, y_pred_v3)

print(f"--- RISULTATI PIPELINE (Scaling + Area Discretizzata) ---")
print(f"R2 Score 3: {r2_v3:.4f}")
print(f"R2 Score 2: {r2_v2:.4f}")
print(f"MSLE 3:     {msle_v3:.4f}")
print(f"MSLE 2:     {msle_v2:.4f}")


--- RISULTATI PIPELINE (Scaling + Area Discretizzata) ---
R2 Score 3: -4.9732
R2 Score 2: -5.0043
MSLE 3:     28.5487
MSLE 2:     37.8575
